In [ ]:
import os
import path_file
os.chdir(os.path.dirname(os.path.dirname(path_file.__file__)))
os.getcwd()

In [ ]:
import sys
import cantera as ct
import numpy as np
import scipy.optimize as opt
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from matplotlib.legend_handler import HandlerTuple

import phases
import feedstock as fs
import fuel as fu
import energy as en
import gasifier as gs

In [ ]:
T0 = 298.15
P0 = ct.one_atm
N = 1.0+1E-5

ERs = np.arange(0.0, N, 0.01)

outs_by_ER1= []
outs_by_ER2= []
outs_by_ER3= []

In [ ]:
#Li et al.
fuel1 = fs.create_fuel_stream(1, 
                              [48.84, 6.81, 35.11, 0.81, 0.21, 0.00], #ultimate db
                              8.22 / 100 , #ash db
                              0/ 100, #moist db
                              HHV=19.63,#HHV MJ/kg, d.b.
                              ashComposition=[5.67, 25.58, 2.21, 0.27, 0, 30.39, 17.17, 12.70, 0, 6.01, 0]) 
print(fuel1.get_mass())
#Chen et al.
fuel2 = fs.create_fuel_stream(1, 
                              [44.3, 5.8, 45.2, 0.7, 0.1, 0.00], #ultimate db
                              3.9 / 100, #ash db
                              0/ 100, #moist db
                              HHV=16.1,#HHV MJ/kg, d.b.
                              ) 
print(fuel2.get_mass())
#Okolie et al.
fuel3 = fs.create_fuel_stream(1, #
                              [41.98, 5.05, 47.46, 0.46, 0.45, 0.00], #ultimate db
                              4.6 / 100, #ash db
                              5.2/ 100, #moist db
                              HHV=16.4,# HHV MJ/kg, d.b.
                              ) 
print(fuel3.get_mass())
leg = ['SS-1 ', 'SS-2 ', 'SS-3 ']

In [ ]:
stoichAir1= fs.create_air_from_ER(fuel1, 1.0)
stoichAirMass1 = stoichAir1.get_mass()
stoichOxygen1 = fs.create_O2_from_ER(fuel1, 1.0)
stoichOxygenMass11 = stoichOxygen1.get_mass()

for i, er in enumerate(ERs):
    air1 = fs.create_air_from_ER(fuel1, er)
    outlet = gs.gasify_nonisot(fuel1, air1, T0=T0, P=P0, heatLossFraction=0, charFormation=0, directMethaneConv=0)
    outs_by_ER1.append(outlet)


In [ ]:
stoichAir2= fs.create_air_from_ER(fuel2, 1.0)
stoichAirMass2 = stoichAir2.get_mass()
stoichOxygen2 = fs.create_O2_from_ER(fuel2, 1.0)
stoichOxygenMass12 = stoichOxygen2.get_mass()

for i, er in enumerate(ERs):
    air2 = fs.create_air_from_ER(fuel2, er)
    outlet = gs.gasify_nonisot(fuel2, air2, T0=T0, P=P0, heatLossFraction=0, charFormation=0, directMethaneConv=0)
    outs_by_ER2.append(outlet)

In [ ]:
stoichAir3= fs.create_air_from_ER(fuel3, 1.0)
stoichAirMass3 = stoichAir3.get_mass()
stoichOxygen3 = fs.create_O2_from_ER(fuel3, 1.0)
stoichOxygenMass13 = stoichOxygen3.get_mass()

for i, er in enumerate(ERs):
    air3 = fs.create_air_from_ER(fuel3, er)
    outlet = gs.gasify_nonisot(fuel3, air3, T0=T0, P=P0, heatLossFraction=0, charFormation=0, directMethaneConv=0)
    outs_by_ER3.append(outlet)

In [ ]:
fig1, ax1a = plt.subplots(figsize=(8, 5), dpi=1000)

ax1a.plot(ERs, [out.T for out in outs_by_ER1], '--',  linewidth=0.7, label=leg[0], color='black')
ax1a.plot(ERs, [out.T for out in outs_by_ER2],'-',  linewidth=0.7, label=leg[1], color='black')
ax1a.plot(ERs, [out.T for out in outs_by_ER3], ':', linewidth=0.7, label=leg[2], color='black')

ax1a.fill_between(ERs, [out.T for out in outs_by_ER1], [out.T for out in outs_by_ER2], color='lightgray', alpha=0.5)
ax1a.fill_between(ERs, [out.T for out in outs_by_ER2], [out.T for out in outs_by_ER3], color='lightgray', alpha=0.5)

ax1a.set_xlabel('ER', fontsize=14)
ax1a.set_ylabel('Temperatura adiabática de equilíbrio (K)', fontsize=14)
ax1a.set_xlim(0.0, 1.0)
ax1a.set_ylim(273, 2300)
#ax1a.grid(True, linestyle='--', alpha=0.7, linewidth=0.7)
ax1a.legend(loc='best', fontsize=14)
ax1a.tick_params(axis='both', which='major', labelsize=12)

#plt.title('Adiabatic Equilibrium Temperature vs. Equivalence Ratio', fontsize=14)

plt.savefig('figures/sensit_Temp_ER.png', bbox_inches='tight')

plt.show()

In [ ]:
fig2_1, ax2_1a = plt.subplots(figsize=(8,5), dpi=1000)
siz = 80
ax2_1a.plot(ERs, [100*out.get_syngas_fraction('CO', water=True, nitrogen=True) for out in outs_by_ER1], '--',  linewidth=0.7, label=leg[0]+'$CO$', color='black')
ax2_1a.plot(ERs, [100*out.get_syngas_fraction('CO', water=True, nitrogen=True) for out in outs_by_ER2], '-',  linewidth=0.7, label=leg[1]+'$CO$', color='black')
ax2_1a.plot(ERs, [100*out.get_syngas_fraction('CO', water=True, nitrogen=True) for out in outs_by_ER3], ':',  linewidth=0.7, label=leg[2]+'$CO$', color='black')

ax2_1a.fill_between(ERs, [100*out.get_syngas_fraction('CO', water=True, nitrogen=True) for out in outs_by_ER1], 
                  [100*out.get_syngas_fraction('CO', water=True, nitrogen=True) for out in outs_by_ER2], color='lightgray', alpha=0.5)
ax2_1a.fill_between(ERs, [100*out.get_syngas_fraction('CO', water=True, nitrogen=True) for out in outs_by_ER2], 
                  [100*out.get_syngas_fraction('CO', water=True, nitrogen=True) for out in outs_by_ER3], color='lightgray', alpha=0.5)

ax2_1a.legend(loc='best', fontsize=14)
ax2_1a.set_xlabel('ER', fontsize=14)
ax2_1a.set_ylabel('Fração molar de $CO$ (%)', fontsize=14)
ax2_1a.set_xlim(0, 1)
ax2_1a.set_ylim(0, siz)

plt.savefig('figures/sensit_Compos_ER_CO.jpg')
plt.show()

In [ ]:
fig2_2, ax2_2a = plt.subplots(figsize=(8,5), dpi=1000)
ax2_2a.plot(ERs, [100*out.get_syngas_fraction('H2', water=True, nitrogen=True) for out in outs_by_ER1], '--',  linewidth=0.7, label=leg[0]+'$H_2$', color='black')
ax2_2a.plot(ERs, [100*out.get_syngas_fraction('H2', water=True, nitrogen=True) for out in outs_by_ER2], '-',  linewidth=0.7, label=leg[1]+'$H_2$', color='black')
ax2_2a.plot(ERs, [100*out.get_syngas_fraction('H2', water=True, nitrogen=True) for out in outs_by_ER3], ':',  linewidth=0.7, label=leg[2]+'$H_2$', color='black')

ax2_2a.fill_between(ERs, [100*out.get_syngas_fraction('H2', water=True, nitrogen=True) for out in outs_by_ER1], 
                  [100*out.get_syngas_fraction('H2', water=True, nitrogen=True) for out in outs_by_ER2], color='lightgray', alpha=0.5)
ax2_2a.fill_between(ERs, [100*out.get_syngas_fraction('H2', water=True, nitrogen=True) for out in outs_by_ER2], 
                  [100*out.get_syngas_fraction('H2', water=True, nitrogen=True) for out in outs_by_ER3], color='lightgray', alpha=0.5)

ax2_2a.legend(loc='best', fontsize=14)
ax2_2a.set_xlabel('ER', fontsize=14)
ax2_2a.set_ylabel('Fração molar de $H_2$ (%)', fontsize=14)
ax2_2a.set_xlim(0, 1)
ax2_2a.set_ylim(0, siz)

plt.savefig('figures/sensit_Compos_ER_H2.jpg')
plt.show()

In [ ]:
fig2_3, ax2_3a = plt.subplots(figsize=(8,5), dpi=1000)
ax2_3a.plot(ERs, [100*out.get_syngas_fraction('CO2', water=True, nitrogen=True) for out in outs_by_ER1], '--',  linewidth=0.7, label=leg[0]+'$CO_2$', color='black')
ax2_3a.plot(ERs, [100*out.get_syngas_fraction('CO2', water=True, nitrogen=True) for out in outs_by_ER2], '-',  linewidth=0.7, label=leg[1]+'$CO_2$', color='black')
ax2_3a.plot(ERs, [100*out.get_syngas_fraction('CO2', water=True, nitrogen=True) for out in outs_by_ER3], ':',  linewidth=0.7, label=leg[2]+'$CO_2$', color='black')

ax2_3a.fill_between(ERs, [100*out.get_syngas_fraction('CO2', water=True, nitrogen=True) for out in outs_by_ER1], 
                 [100*out.get_syngas_fraction('CO2', water=True, nitrogen=True) for out in outs_by_ER2], color='lightgray', alpha=0.5)
ax2_3a.fill_between(ERs, [100*out.get_syngas_fraction('CO2', water=True, nitrogen=True) for out in outs_by_ER2], 
                  [100*out.get_syngas_fraction('CO2', water=True, nitrogen=True) for out in outs_by_ER3], color='lightgray', alpha=0.5)

ax2_3a.legend(loc='best', fontsize=14)
ax2_3a.set_xlabel('ER', fontsize=14)
ax2_3a.set_ylabel('Fração molar de $CO_2$ (%)', fontsize=14)
ax2_3a.set_xlim(0, 1)
ax2_3a.set_ylim(0, siz)

plt.savefig('figures/sensit_Compos_ER_CO2.jpg')
plt.show()

In [ ]:
fig2_4, ax2_4a = plt.subplots(figsize=(8,5), dpi=1000)
ax2_4a.plot(ERs, [100*out.get_syngas_fraction('CH4', water=True, nitrogen=True) for out in outs_by_ER1], '--',  linewidth=0.7, label=leg[0]+'$CH_4$', color='black')
ax2_4a.plot(ERs, [100*out.get_syngas_fraction('CH4', water=True, nitrogen=True) for out in outs_by_ER2], '-',  linewidth=0.7, label=leg[1]+'$CH_4$', color='black')
ax2_4a.plot(ERs, [100*out.get_syngas_fraction('CH4', water=True, nitrogen=True) for out in outs_by_ER3], ':',  linewidth=0.7, label=leg[2]+'$CH_4$', color='black')

ax2_4a.fill_between(ERs,  [100*out.get_syngas_fraction('CH4', water=True, nitrogen=True) for out in outs_by_ER1], 
                 [100*out.get_syngas_fraction('CH4', water=True, nitrogen=True) for out in outs_by_ER2], color='lightgray', alpha=0.5)
ax2_4a.fill_between(ERs,  [100*out.get_syngas_fraction('CH4', water=True, nitrogen=True) for out in outs_by_ER2], 
                  [100*out.get_syngas_fraction('CH4', water=True, nitrogen=True) for out in outs_by_ER3], color='lightgray', alpha=0.5)

ax2_4a.legend(loc='best', fontsize=14)
ax2_4a.set_xlabel('ER', fontsize=14)
ax2_4a.set_ylabel('Fração molar de $CH_4$ (%)', fontsize=14)
ax2_4a.set_xlim(0, 1)
ax2_4a.set_ylim(0, siz)
plt.savefig('figures/sensit_Compos_ER_CH4.jpg')
plt.show()

In [ ]:
fig2_5, ax2_5a = plt.subplots(figsize=(8,5), dpi=1000)
ax2_5a.plot(ERs, [100*out.get_syngas_fraction('N2', water=True, nitrogen=True) for out in outs_by_ER1], '--',  linewidth=0.7, label=leg[0]+'$N_2$', color='black')
ax2_5a.plot(ERs, [100*out.get_syngas_fraction('N2', water=True, nitrogen=True) for out in outs_by_ER2], '-',  linewidth=0.7, label=leg[1]+'$N_2$', color='black')
ax2_5a.plot(ERs, [100*out.get_syngas_fraction('N2', water=True, nitrogen=True) for out in outs_by_ER3], ':',  linewidth=0.7, label=leg[2]+'$N_2$', color='black')

ax2_5a.fill_between(ERs, [100*out.get_syngas_fraction('N2', water=True, nitrogen=True) for out in outs_by_ER1], 
                  [100*out.get_syngas_fraction('N2', water=True, nitrogen=True) for out in outs_by_ER2], color='lightgray', alpha=0.5)
ax2_5a.fill_between(ERs, [100*out.get_syngas_fraction('N2', water=True, nitrogen=True) for out in outs_by_ER2], 
                  [100*out.get_syngas_fraction('N2', water=True, nitrogen=True) for out in outs_by_ER3], color='lightgray', alpha=0.5)

ax2_5a.legend(loc='best', fontsize=14)
ax2_5a.set_xlabel('ER', fontsize=14)
ax2_5a.set_ylabel('Fração molar de $N_2$ (%)', fontsize=14)
ax2_5a.set_xlim(0, 1)
ax2_5a.set_ylim(0, siz)

plt.savefig('figures/sensit_Compos_ER_N2.jpg')
plt.show()

In [ ]:
fig2_6, ax2_6a = plt.subplots(figsize=(8,5), dpi=1000)
ax2_6a.plot(ERs, [100*out.get_syngas_fraction('H2O', water=True, nitrogen=True) for out in outs_by_ER1], '--',  linewidth=0.7, label=leg[0]+'$H_2O$', color='black')
ax2_6a.plot(ERs, [100*out.get_syngas_fraction('H2O', water=True, nitrogen=True) for out in outs_by_ER2], '-',  linewidth=0.7, label=leg[1]+'$H_2O$', color='black')
ax2_6a.plot(ERs, [100*out.get_syngas_fraction('H2O', water=True, nitrogen=True) for out in outs_by_ER3], ':',  linewidth=0.7, label=leg[2]+'$H_2O$', color='black')

ax2_6a.fill_between(ERs, [100*out.get_syngas_fraction('H2O', water=True, nitrogen=True) for out in outs_by_ER1], 
                  [100*out.get_syngas_fraction('H2O', water=True, nitrogen=True) for out in outs_by_ER2], color='lightgray', alpha=0.5)
ax2_6a.fill_between(ERs, [100*out.get_syngas_fraction('H2O', water=True, nitrogen=True) for out in outs_by_ER2], 
                  [100*out.get_syngas_fraction('H2O', water=True, nitrogen=True) for out in outs_by_ER3], color='lightgray', alpha=0.5)

ax2_6a.legend(loc='best', fontsize=14)
ax2_6a.set_xlabel('ER', fontsize=14)
ax2_6a.set_ylabel('Fração molar de $H_2O$ (%)', fontsize=14)
ax2_6a.set_xlim(0, 1)
ax2_6a.set_ylim(0, siz)

plt.savefig('figures/sensit_Compos_ER_H2O.jpg')
plt.show()

In [ ]:
fig3_1, ax3_1a = plt.subplots(figsize=(8,5), dpi=1000) 
    
ax3_1a.set_xlim(0, 1)
ax3_1a.set_ylim(0, 40)

ax3_1a.plot(ERs, [out.get_syngas_hhv(basis='vol', water=False, nitrogen=True) for out in outs_by_ER1], '--',  linewidth=0.7, label=leg[0]+'$PCS$', color='black')
ax3_1a.plot(ERs, [out.get_syngas_hhv(basis='vol', water=False, nitrogen=True) for out in outs_by_ER2], '-',  linewidth=0.7, label=leg[1]+'$PCS$', color='black')
ax3_1a.plot(ERs, [out.get_syngas_hhv(basis='vol', water=False, nitrogen=True) for out in outs_by_ER3], ':',  linewidth=0.7, label=leg[2]+'$PCS$', color='black')

ax3_1a.fill_between(ERs, [out.get_syngas_hhv(basis='vol', water=False, nitrogen=True) for out in outs_by_ER1], 
                  [out.get_syngas_hhv(basis='vol', water=False, nitrogen=True) for out in outs_by_ER2], color='lightgray', alpha=0.5)
ax3_1a.fill_between(ERs, [out.get_syngas_hhv(basis='vol', water=False, nitrogen=True) for out in outs_by_ER2], 
                  [out.get_syngas_hhv(basis='vol', water=False, nitrogen=True) for out in outs_by_ER3], color='lightgray', alpha=0.5)

ax3_1a.legend(loc='best', fontsize=14)
ax3_1a.set_xlabel('ER', fontsize=14)
ax3_1a.set_ylabel('Poder Calorífico Superior, PCS(MJ/Nm³)', fontsize=14)

plt.savefig('figures/sensit_Eff_ER_HHV.jpg')
plt.show()

In [ ]:
fig3_2, ax3_2a = plt.subplots(figsize=(8,5), dpi=1000)
    
ax3_2a.set_xlim(0, 1)
ax3_2a.set_ylim(0, 10)
    
ax3_2a.set_xlabel('ER', fontsize=14)
ax3_2a.set_ylabel('Rendimento de Syngas (Nm³/kg)', fontsize=14)

ax3_2a.plot(ERs, [out.get_syngas_amount(basis='vol', water=False, nitrogen=True) for out in outs_by_ER1], '--',  linewidth=0.7, label=leg[0], color='black')
ax3_2a.plot(ERs, [out.get_syngas_amount(basis='vol', water=False, nitrogen=True) for out in outs_by_ER2], '-',  linewidth=0.7, label=leg[1], color='black')
ax3_2a.plot(ERs, [out.get_syngas_amount(basis='vol', water=False, nitrogen=True) for out in outs_by_ER3], ':',  linewidth=0.7, label=leg[2], color='black')

ax3_2a.fill_between(ERs, [out.get_syngas_amount(basis='vol', water=False, nitrogen=True) for out in outs_by_ER1], 
                  [out.get_syngas_amount(basis='vol', water=False, nitrogen=True) for out in outs_by_ER2], color='lightgray', alpha=0.5)
ax3_2a.fill_between(ERs, [out.get_syngas_amount(basis='vol', water=False, nitrogen=True) for out in outs_by_ER2], 
                  [out.get_syngas_amount(basis='vol', water=False, nitrogen=True) for out in outs_by_ER3], color='lightgray', alpha=0.5)

ax3_2a.legend(loc='best', fontsize=14)
plt.savefig('figures/sensit_Eff_ER_Yield.jpg')
plt.show()

In [ ]:
fig3_3, ax3_3a = plt.subplots(figsize=(8,5), dpi=1000) 
    
ax3_3a.set_xlim(0, 1)
ax3_3a.set_ylim(0, 90)

ax3_3a.plot(ERs, [100*out.get_syngas_CGE() for out in outs_by_ER1], '--',  linewidth=0.7, label=leg[0], color='black')
ax3_3a.plot(ERs, [100*out.get_syngas_CGE() for out in outs_by_ER2], '-',  linewidth=0.7, label=leg[1], color='black')
ax3_3a.plot(ERs, [100*out.get_syngas_CGE() for out in outs_by_ER3], ':',  linewidth=0.7, label=leg[2], color='black')

ax3_3a.fill_between(ERs, [100*out.get_syngas_CGE() for out in outs_by_ER1], 
                  [100*out.get_syngas_CGE() for out in outs_by_ER2], color='lightgray', alpha=0.5)
ax3_3a.fill_between(ERs, [100*out.get_syngas_CGE() for out in outs_by_ER2], 
                  [100*out.get_syngas_CGE() for out in outs_by_ER3], color='lightgray', alpha=0.5)

ax3_3a.legend(loc='best', fontsize=14)
ax3_3a.set_xlabel('ER', fontsize=14)
ax3_3a.set_ylabel('Eficiência de gás frio (%)', fontsize=14)

plt.savefig('figures/sensit_Eff_ER_CGE.jpg')
plt.show()